In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [75]:

class RDF:

    # Initialises the class 
    # parameters
    #     wavefunction: the wave function to use , 
    #     start : start of the range (min x)
    #     end   : end of the range (max x)
    def __init__(self, wave_function, start, end):  
        self.wave_function = wave_function
        self.start = start
        self.end = end

    # Compute the normalised y values such that area under the curve is 1.
    # The integral is computed using an increasing number of intervals until the difference 
    # between the current value and the previous falls below the specified converge threshold.
    # Parameters
    #     integration_method:   integration method, must be 'mid' or 'trap'
    #     convergence_threshold: value below which the integral is considered to have converged
    #     initial_intervals: initial number of intervals
    #     interval_increment: number of intervals to add when refining the 
    # Return {normalised y-values, normalisation factor, x-values, number of intervals}
    def normalise_area(self, integration_method, convergence_threshold, initial_intervals, interval_increment): 
        area, x_values, y_values, intervals = self.__converge_interval(integration_method, convergence_threshold, initial_intervals, interval_increment) 
        norm_y_values = []
        normalization_factor = 1 / area
        for y in y_values:
            norm_y_values.append(y * normalization_factor)
        return norm_y_values, normalization_factor, x_values, intervals

    # Computes normalised y values such that their maximum individual value is 1.
    # Parameters:
    #     intervals: number of intervals (this will result in `intervals + 1` values)
    # Return {normalised y-values, normalisation factor, x-values}
    def normalise_height(self, intervals):  
        x_values = self.__linearPoints(intervals)
        y_values = self.__values_range(x_values)
        norm_y_values = []
        normalization_factor = 1 / max(y_values)
        for i in y_values:
            norm_y_values.append(i * normalization_factor)
        return norm_y_values, normalization_factor, x_values 

    # Compute an integral using an increasing number of intervals until the difference 
    # between the current value and the previous falls below a specified converge threshold.
    # Parameters
    #     integral_method:   integration method, must be 'mid' or 'trap'
    #     convergence_threshold: value below which the integral is considered to have converged
    #     initial_intervals: initial number of intervals
    #     interval_increment: number of intervals to add when refining the integral value
    # Return {area under the curve, x-values, y-values, number of intervals}
    def __converge_interval(self, integral_method, converge_threshold, initial_intervals, interval_change):
        area_prev, x_values, y_values = self.__integrate(integral_method, initial_intervals)
        intervals = initial_intervals
        epsilon = converge_threshold * 2
        while epsilon > converge_threshold:
            intervals += interval_change
            area_current, x_values, y_values = self.__integrate(integral_method, intervals)
            epsilon = abs(area_current - area_prev)
            area_prev = area_current
        return area_current, x_values, y_values, intervals

    # Computes the integral using the specificed method 
    # Parameters:
    #   method: the method of integration 'mid' or 'trap'
    #   intervals: the number of intervals to use 
    # Returns { integral value (area under the curve), x-values, y-values}
    def __integrate(self, method, intervals): 
        length = (self.end - self.start)/intervals
        if method == 'mid':
            x_values = self.__MidPoints(intervals)
            y_values = self.__values_range(x_values) 
            integral = length*(sum(y_values))
            return integral, x_values, y_values
        elif method == 'trap':
            x_values = self.__linearPoints(intervals)
            y_values = self.__values_range(x_values)
            integral = length*(1/2)*(2*sum(y_values)-y_values[0]-y_values[-1])
            return integral, x_values , y_values
        else:
            raise Exception("Bad integration method, should be 'mid' or 'trap'.")
        
    # Computes the RDF of the wave function at a given number of points 
    # Parameters:
    #   x_values: x coordinates (radius) of the points to calculate
    def __values_range(self, x_values):
        y_values = []
        for radius in x_values:
            four_pi_radius_squared = 4 * np.pi * (radius ** 2)
            y = four_pi_radius_squared * ((self.wave_function(radius)) ** 2)
            y_values.append(y)
        return y_values

    # Calculates the x-values for the mid-point rectangle integration method
    # Parameters: 
    #   intervals: number of intervals (will result in `intervals` x-values)
    # Returns {x-values}
    def __MidPoints(self, intervals): 
        interval_length = (self.end - self.start) / intervals
        points = []
        x = self.start
        for i in range(intervals):
            points.append((x*2 + interval_length)/2)
            x = x + interval_length 
        return points
        
    # Calculates the x coordinates for Trapezoidal integration method 
    # Parameters: 
    #   intervals: number of intervals (will result in `intervals + 1` x-values)
    # Returns {x-values}
    def __linearPoints(self, intervals): 
        interval_length = (self.end - self.start) / intervals 
        points = []
        for i in range(intervals + 1):
            points.append(self.start + interval_length*i)
        return points

# wave functions

def psi_1s(radius):
    return np.exp((-radius)/2)
    
def psi_2s(radius):
    return ((2 - radius) * np.exp(-radius / 2))

def psi_2p(radius):
    return ((6 - 6 * radius + (radius**2)) * np.exp(-radius / 2))

def psi_3p(radius):
    return ((4 * radius - radius**2)*np.exp(-radius / 2))

def psi_3d(radius):
    return ((radius**2)*np.exp(-radius / 2))

